<h1>System of Linear Equations Routines</h1>

In [1]:
# Let us first define the numbers we will work with. Note the use of KK when defining
# both matrices and vectors. Details aside it will make output numbers look nice if you set
# KK equal QQ. Other possible options are RR, CC, QQ, QQbar; here QQbar is another nice option
KK = QQ

def GetEch(M265Matrix):
    # given a matrix defined as above the function returns the echelon form of the matrix
    return M265Matrix.echelon_form()

def HomSol(M265Matrix):
    # obtain the homogeneous solution set of SLE with matrix M265Matrix
    return M265Matrix.right_kernel_matrix()

def ParSol(M265Matrix,M265Vector):
    # obtain a particular solutino to SLE with matrix M265Matrix and vector of constant M265Vector
    (rw,cl)=M265Matrix.dimensions()
    if(rw!=len(M265Vector)):
        print("dimension error")
    try:
        return M265Matrix.solve_right(M265Vector)
    except ValueError:
        #if ValueError occurs there is no solution but instead of raising the error we return
        # the empty set
        return vector(KK,[])
    return False
    
def GetERO(M265Matrix,debug=False):
    # get one list of elemetary operations that reduce a matrix to Reduced Echelon Form
    resEliminations = []
    [Rw,Cl]=M265Matrix.dimensions()
    M265copy = Matrix(KK,M265Matrix)
    if (debug):
        print(M265copy)
    rw = 0
    cl = 0
    for c in range(Cl):
        hasPivot = False
        # first we check if the column under question has a pivot
        for r in range(rw,Rw):
            if (M265copy[r][cl]!=0):
                hasPivot = True
                if (r!=rw):
                    EO = identity_matrix(KK,Rw)
                    EO.swap_rows(rw,r)
                    resEliminations.append(EO)
                    M265copy.swap_rows(rw,r)
                    if (debug):
                        print("\n")
                        print("Swap row ",rw,"with row", r)
                        print(M265copy)
                break # if there is a pivot 
        if (hasPivot):
            # if there is a pivot first scale it to one if the pivot is not one
            if (M265copy[rw][cl]!= KK(1)):
                EO=identity_matrix(KK,Rw)
                EO.rescale_row(rw,KK(1)/M265copy[rw][cl])
                resEliminations.append(EO)
                M265copy=resEliminations[-1]*M265copy
                if (debug):
                    print("\n")
                    print("Scale row", rw)
                    print(M265copy)

            # make the respective entries in the column zero except for the row that contains the pivot (r==rw)
            # further if a row has already entry zero at that column skip that row as the resulting scaling
            # is simply the identity matrix (M265copy[r][cl]==0)
            for r in range(Rw):
                if ((r==rw) or (M265copy[r][cl]==0)):
                    continue
                EO = identity_matrix(KK,Rw)
                EO.add_multiple_of_row(r,rw,-M265copy[r][cl])
                resEliminations.append(EO)
                #M265copy = M265copy.add_multiple_of_row(r,rw,-M265copy[r][cl])
                M265copy=resEliminations[-1]*M265copy
                if(debug):
                    print("\n")
                    print("M265copy multiple of row",rw,"to row",r )
                    print(M265copy)
            rw += 1 # if there was a pivot in this column for the remaining rows up we search for pivots
            # only on rows that are below it
        cl += 1
    return resEliminations

<h1>Homework</h1>

<h2>Restrictions</h2>
<p>You may run the sage kernel or any other kernel of your choice. However, beyond the Standard Routines provided at the top of this notebook no other function calls are allowed from the sage (or any other library).</p>
<p>If you cannot run sage, please update the above routines with the solutions I've provided for the first homework and use those instead. You are welcome to contact me for further information if that is the case.</p>

<h2>Linear dependence</h2>
In the homework $M$ denotes a square matrix of order $d$ and  $\vec{v} = vv$ a vector with $d$ components that is different from the zero vector (i.e. $\vec{v}\ne \vec{0}$).

Consider the sequence $\vec{v}_0 = \vec{v}$, $\vec{v}_1 = M\vec{v}_0\dots$ defined recursively $$\vec{v}_{i+1} = M\vec{v}_i$$. Since all vectors lie in the same $d$-dimensional vector space then there is a smallest integer $k$ such that the set $\vec{v}_0,\dots\vec{v}_{k}$ is linearly dependent but any set  $\vec{v}_0,\dots\vec{v}_{j}$ with $j<k$ is linearly independent. With that in mind it is necessarily the case that in the equation $$\alpha_0\vec{v}_0 + \alpha_1\vec{v}_1+\cdots+\alpha_{k-1}\vec{v}_{k-1}+\alpha_{k}\vec{v}_k =\vec{0}$$ the coefficient $\alpha_k\ne 0$.

Your tasks are:
<dl>
    <dt>DepDegree(M,vv)</dt>
    <dd>Given a matrix $M$ and a vector $vv$ return the value $k$ described above.</dd> 
    <dt>DepCoefs(M,vv)</dt>
    <dd>Given a matrix $M$ and a vector $vv$ return the values $\frac{\alpha_0}{\alpha_k},\dots,\frac{\alpha_{k-1}}{\alpha_k}$ in a list.</dd>
</dl>
<h2>Remarks</h2>
When you program your routines your routines <em>must not</em> use Python's print function in <em>any</em> form. Your functions should simply return the above values.

In [2]:
def DepDegree(MM, vv):
    res = 0
    vectors = [vv]
    coefficient_matrix = Matrix(QQ, vectors)
    
    while not HomSol(coefficient_matrix.transpose()): #coefficient_matrix.transpose().right_kernel_matrix():
        vectors.append(MM * vectors[-1])
        coefficient_matrix = Matrix(QQ, vectors)
        res += 1

    return res


def DepCoefs(MM, vv):
    count = 0
    vectors = [vv]
    coefficient_matrix = Matrix(QQ, vectors)
    
    while not HomSol(coefficient_matrix.transpose()): # coefficient_matrix.transpose().right_kernel_matrix():
        vectors.append(MM * vectors[-1])
        coefficient_matrix = Matrix(QQ, vectors)
        count += 1
        
    # print(coefficient_matrix.transpose())
    
    return coefficient_matrix.transpose().right_kernel_matrix(basis = "pivot") # coefficient_matrix.transpose().solve_right(constant_vector)

<h2>Finding eigenvalue and associated eigenspace</h2>
Following the previous homework you need to implement the following tasks:
<dl>
    <dt><code>MatrixDeterminant(M)</code></dt>
    <dd>Given a square matrix $M$ return the determinant of $M$</dd>
    <dt><code>EigenValue(M)</code></dt>
    <dd>Given a matrix $M$ compute one eigenvalue for the matrix $M$.</dd> 
    <dt><code>TestValue(M,lmbd)</code></dt>
    <dd>Return True if <code>lmbd</code> is an eigenvalue for $M$ and False otherwise</dd>  
    <dt><code>EigenSpace(M,lmbd)</code></dt>
    <dd>Given a matrix $M$ and a value <code>lmbd</code> return a list of basis vectors for the eigenspace associated with <code>lmbd</code>.</dd>
</dl>

In [3]:
import random

def MatrixDeterminant(M, debug: bool = False):
    res = KK(1)
    [Rw,Cl]= M.dimensions()
    M265copy = Matrix(KK,M)
    if (debug):
        print(M265copy)
    rw = 0
    cl = 0
    for c in range(Cl):
        hasPivot = False
        # first we check if the column under question has a pivot
        for r in range(rw,Rw):
            if (M265copy[r][cl]!=0):
                hasPivot = True
                if (r!=rw):
                    res *= -1
                    M265copy.swap_rows(rw,r)
                    if (debug):
                        print("\n")
                        print("Swap row ",rw,"with row", r)
                        print(M265copy)
                break # if there is a pivot 
        if (hasPivot):
            # if there is a pivot first scale it to one if the pivot is not one
            if ((ratio := M265copy[rw][cl]) != KK(1)):
                res *= KK(1) / ratio
                # EO=identity_matrix(KK,Rw)
                # EO.rescale_row(rw,KK(1)/M265copy[rw][cl])
                M265copy.rescale_row(rw, KK(1) / ratio)
                if (debug):
                    print("\n")
                    print("Scale row", rw)
                    print(M265copy)

            # make the respective entries in the column zero except for the row that contains the pivot (r==rw)
            # further if a row has already entry zero at that column skip that row as the resulting scaling
            # is simply the identity matrix (M265copy[r][cl]==0)
            for r in range(Rw):
                if ((r==rw) or (M265copy[r][cl]==0)):
                    continue
                EO = identity_matrix(KK,Rw)
                EO.add_multiple_of_row(r,rw,-M265copy[r][cl])
                #M265copy = M265copy.add_multiple_of_row(r,rw,-M265copy[r][cl])
                M265copy= EO * M265copy
                if(debug):
                    print("\n")
                    print("M265copy multiple of row",rw,"to row",r )
                    print(M265copy)
            rw += 1 # if there was a pivot in this column for the remaining rows up we search for pivots
            # only on rows that are below it
        cl += 1
        
    
    zero_row = vector(QQ, [0 for _ in range(Cl)])
    
    if M265copy[-1] != zero_row:
        return 1 / res
    else:
        return 0
    
def derivative(coefs):
    result = []
    
    for i, entry in enumerate(coefs[0]):
        if i != 0:
            result.append(entry * i)
    
    return Matrix(QQ, result)
    

def divisors(x):
    result = []
    for i in range(1, abs(x) + 1):
        if x % i == 0:
            result.append(i)
            
    return result

def rational_roots(coefs):
    roots = []
    count = 0

    while coefs.dimensions()[1] > 1:
        for i in range(coefs.dimensions()[1] - 1, -1, -1):
            print(f"{coefs[0][i]}x^{i}", end = ' ')
        print()
 
        if coefs[0][0].denominator() != 1 or coefs[0][-1].denominator() != 1:
            if coefs[0][0].denominator() != 1:
                coefs *= coefs[0][0].denominator()
            if coefs[0][-1].denominator() != 1:
                coefs *= coefs[0][-1].denominator()
            # print(coefs)
                
        div_a0 = divisors(coefs[0][0])
        div_an = divisors(coefs[0][-1])
        
        combinations = []
        
        for a in div_a0:
            for b in div_an:
                possible = KK(a / b)
                if possible not in combinations:
                    combinations.append(KK(a / b))
                    combinations.append(KK(-a / b))
                    
        if coefs[0][0] == 0:
            if count == 0 or (count != 0 and 0 in roots):
                roots.append(0)
        # print(combinations)
        for c in combinations:
            res = 0
            for i, entry in enumerate(coefs[0]):
                res += entry * (c**i)
            if res == 0 and count == 0:
                roots.append(c)
            elif res == 0 and c in roots:
                # print("Found multiple roots")
                roots.append(c)
   
        count += 1
        coefs = derivative(coefs)
        
    return roots
    
    
def EigenValue(M):
    row, column = M.dimensions()
    zv = vector(QQ, [0 for _ in range(column)])
    rv = random_vector(QQ, column)
    
    while rv == zv:                     # ensure to have non-zero vector
        rv = random_vector(QQ, column)
    
    coefficients = DepCoefs(M, rv)

    roots = rational_roots(coefficients)
    index = randint(0, len(roots) - 1)
    
    return roots
  
    
def TestValue(M, lmbd):
    ii = identity_matrix(QQ, M.dimensions()[0])
    # zv = vector(QQ, [0 for _ in range(M.dimensions()[1])])
    result = (M - lmbd * ii)
    sol = HomSol(result)
    
    if not sol:
        return False
    else:
        return True


def EigenSpace(M, lmbd):
    if TestValue(M, lmbd):
        ii = identity_matrix(QQ, M.dimensions()[0])
        result = (M - lmbd * ii)
        return HomSol(result)

In [6]:
M = Matrix(QQ, [[-67, 116, 48], [-25, 44, 18], [-35, 59, 25]])
v0 = vector(QQ, [2, 2, 3])
M.eigenvectors_right()

[(3,
  [
  (1, 1/2, 1/4)
  ],
  1),
 (1,
  [
  (1, 1, -1)
  ],
  1),
 (-2,
  [
  (1, 1/4, 3/4)
  ],
  1)]

In [11]:
EigenSpace(M, 3)

1x^3 -2x^2 -5x^1 6x^0 
3x^2 -4x^1 -5x^0 
6x^1 -4x^0 


[400/7 200/7 100/7]